## Initialization and Imports

In [ ]:
%uv pip install -r requirements.txt

In [ ]:
import ee
from dask.distributed import Client
import dask.distributed
import dask.bag as db
import numpy as np
import pandas as pd
from google.api_core import retry
import requests
import io

# Start a Dask client for parallel computation
client = Client()
client

In [ ]:
!earthengine authenticate

## Load data acquisition and EDA step

# Change bigdata-ahhcash to your project name

In [ ]:
ee.Initialize(project='bigdata-ahhcash')

In [ ]:
def get_label_image() -> ee.Image:
    # Remap the ESA classifications into the Dynamic World classifications
    fromValues = [10, 20, 30, 40, 50, 60, 70, 80, 90, 95, 100]
    toValues = [1, 5, 2, 4, 6, 7, 8, 0, 3, 3, 7]
    return (
        ee.Image("ESA/WorldCover/v100/2020")
        .select("Map")
        .remap(fromValues, toValues)
        .rename("landcover")
        .unmask(0)  # fill missing values with 0 (water)
        .byte()  # 9 classifications fit into an unsinged 8-bit integer
    )

In [ ]:
land_cover_image = get_label_image()

In [ ]:
# Simplified polygons covering most land areas in the world.
WORLD_POLYGONS = [
    # Americas
    [(-33.0, -7.0), (-55.0, 53.0), (-166.0, 65.0), (-68.0, -56.0)],
    # Africa, Asia, Europe
    [
        (74.0, 71.0),
        (166.0, 55.0),
        (115.0, -11.0),
        (74.0, -4.0),
        (20.0, -38.0),
        (-29.0, 25.0),
    ],
    # Australia
    [(170.0, -47.0), (179.0, -37.0), (167.0, -12.0), (128.0, 17.0), (106.0, -29.0)],
]

# Function to get stratified sample points
def get_stratified_sample_coords(land_cover_image, n_per_class_per_region):
    sample_points = []
    region = ee.Geometry.MultiPolygon(WORLD_POLYGONS)
    # for class_value in range(1, 11):  # Adjust based on land cover classification
    #     class_region = land_cover_image.eq(class_value).And(ee.Image.pixelArea().gt(0)).clip(region)
    points = land_cover_image.stratifiedSample(
        numPoints=n_per_class_per_region, 
        # classBand=land_cover_image.bandNames().get(0), 
        region=region, 
        scale=300,  # Adjust scale as needed
        geometries=True
    )
    sample_points.extend(points.aggregate_array('.geo').getInfo())
    return sample_points


In [ ]:
land_cover_image

In [ ]:
start_date = '2013-03-01'
end_date = '2021-12-01'
dates = pd.date_range(start=start_date, end=end_date, freq='MS').strftime('%Y-%m-%d').tolist()
err_image = None
from google.api_core import retry

@retry.Retry(deadline=10 * 60)
def process_and_export(point, dates):
    global err_image
    ee.Initialize(project='bigdata-ahhcash', opt_url='https://earthengine-highvolume.googleapis.com')
    lon, lat = point['coordinates']
    point_geometry = ee.Geometry.Point(lon, lat)

    time_series_data = []
    for date in dates:
        try:
            # Landsat 8 processing
            # dask.distributed.print((date, (lon, lat)))
            landsat_collection = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
                .filterBounds(point_geometry) \
                .filterDate(date, ee.Date(date).advance(1, 'month'))
            landsat_image = landsat_collection.median()
            landsat_ndvi = landsat_image.normalizedDifference(['SR_B5', 'SR_B4']).rename('NDVI')
    
            # MODIS processing (assuming MOD13Q1)
            modis_collection = ee.ImageCollection('MODIS/006/MOD13Q1') \
                .filterBounds(point_geometry) \
                .filterDate(date, ee.Date(date).advance(1, 'month'))
            modis_image = modis_collection.median()
            modis_ndvi = modis_image.select('NDVI')
    
            combined_ndvi = ee.Image.cat([landsat_ndvi, modis_ndvi])
    
            def download_as_numpy(url):
                response = requests.get(url)
                # If we get "429: Too Many Requests" errors, it's safe to retry the request.
                # The Retry library only works with `google.api_core` exceptions.
                if response.status_code == 429:
                    raise exceptions.TooManyRequests(response.text)
                response.raise_for_status()  # Raise an error for bad status codes
                return np.load(io.BytesIO(response.content))
        
            numpy_data = download_as_numpy(combined_ndvi.getDownloadURL(params={
                'region': point_geometry.buffer(5000).bounds().getInfo()['coordinates'],
                'scale': 1000,  # Adjust scale as needed
                'format': 'NPY'
            }))
    
            _ = combined_ndvi.bandNames().getInfo()
            # Add date information to the NumPy array
            # numpy_data = np.expand_dims(numpy_data, axis=0)  # Add a time dimension
            # numpy_data = np.insert(numpy_data, 0, date, axis=0)  # Insert date at the beginning
    
            data_with_date = (date, numpy_data)
            time_series_data.append(data_with_date)
        except Exception as e:
            dask.distributed.print(date)
            continue

    # Extract metadata from the first image
    projection = None
    try:
        projection = combined_ndvi.projection().getInfo()
        crs = projection['crs']
        transform = projection['transform']
        scale_x = transform[0]
        scale_y = -transform[4]
    
        # Create a DataFrame with metadata
        metadata_df = pd.DataFrame({
            'longitude': [lon],
            'latitude': [lat],
            'crs': [crs],
            'scale_x': [scale_x],
            'scale_y': [scale_y]
        })
    
        # Save time series data and metadata
        time_series_array = np.concatenate(time_series_data, axis=0)  # Combine into a single array
        np.savez_compressed(f'stratified_data/time_series_{lon}_{lat}.npz', data=time_series_array)
        metadata_df.to_csv(f'stratified_data/metadata_{lon}_{lat}.csv', index=False)
    except Exception as e:
        dask.distributed.print("error occurred outside the for loop")
    

# Generate stratified sample points
sample_points = get_stratified_sample_coords(land_cover_image, 2) 

# Parallelize processing using Dask
sample_points_bag = db.from_sequence(sample_points)
results = sample_points_bag.map(process_and_export, dates)  # Pass dates to the function
results.compute()

print("Dask operations completed.")

In [ ]:
coordinates = [-119.1548600183745, 53.9257517606046]
lon, lat = coordinates
ee.Initialize(project='bigdata-ahhcash', opt_url='https://earthengine-highvolume.googleapis.com')
start_date = '2013-03-01'
end_date = '2021-12-01'
dates = pd.date_range(start=start_date, end=end_date, freq='MS').strftime('%Y-%m-%d').tolist()

for date in dates:
    try:
        point_geometry = ee.Geometry.Point(lon, lat)
        landsat_collection = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
            .filterBounds(point_geometry) \
            .filterDate(ee.Date(date), ee.Date(date).advance(1, 'month').advance(-1, 'day'))
        landsat_image = landsat_collection.median()
        landsat_ndvi = landsat_image.normalizedDifference(['SR_B5', 'SR_B4']).rename('NDVI')
        print(landsat_ndvi.bandNames().getInfo())
    except Exception as e:
        print(e)
        print(date)

In [30]:
print(ee.Date('2013-12-01').advance(1, 'month').format().getInfo())

2014-01-01T00:00:00
